# Working with our SQL Database

## Prior to running this file you must have run the "Cleaning_Data.ipynb" file and have loaded the two exported csv files into a database on your local pgadmin server

In [1]:
# Importing library
import pandas as pd
import psycopg2
import csv

# Connecting to db
con = psycopg2.connect(
    host = "localhost",
    database = "The_Election_And_COVID", # USE YOUR OWN DATABASE NAME HERE
    user= "postgres", # USE YOUR OWN USER NAME HERE
    password= "PASSWORD", # USE YOUR OWN PASSWORD HERE
    port= 5432 # USE YOUR OWN PORT HERE - DEFAULT IS 5432
)

# Activate connection cursor
cur = con.cursor()

# Executing query
sql_query = '''SELECT * \
FROM covid_counties \
LEFT JOIN \
    (SELECT state_votes.state, \
        CASE WHEN republican_votes > democrat_votes \
        THEN 'Republican Win' \
        ELSE 'Democrat Win' END AS "Winning_party" \
    FROM(\
        SELECT rep.state, \
            republican_votes, \
            democrat_votes \
        FROM (SELECT state, \
            SUM(total_votes) as republican_votes \
            FROM election_results \
            WHERE party = 'REP' \
            GROUP BY state) as rep \
        JOIN \
            (SELECT state, \
            SUM(total_votes) as democrat_votes \
            FROM election_results \
            WHERE party = 'DEM' \
            GROUP BY state) AS dem \
        ON rep.state = dem.state \
        ORDER BY rep.state) AS state_votes) AS state_winner \
ON covid_counties.state = state_winner.state;'''

# Execute the query
cur.execute(sql_query)

# Get the results
results = cur.fetchall()

# Converting the results into a dataframe
df = pd.DataFrame(results)

# Giving the dataframe column headers
df.columns = ["id",
             "date",
             "county",
             "state",
             "fips",
             "cases",
             "deaths",
              "stay_at_home_announced",
              "stay_at_home_effective",
              "total_population",
              "population_density_per_sqmi",
              "percent_single_parent_households_CHR",
              "percent_adults_with_diabetes",
              "percent_food_insecure", 
              "percent_insufficient_sleep", 
              "percent_unemployed_CDC", 
              "percent_no_highschool_diploma", 
              "percent_age_65_and_older", 
              "percent_age_17_and_younger", 
              "percent_disabled",
              "percent_multi_unit_housing", 
              "percent_fair_or_poor_health", 
              "percent_not_proficient_in_english", 
              "percent_female", 
              "percent_rural",
              "percent_minorities",
              "state",
              "Winning_party"
             ]

# Exporting the dataframe as a CSV
df.to_csv(r"C:\Users\Asus\Desktop\Data\Clean_Data\final_database.csv", index=False)
        
# Closing the cursor -- IMPORTANT --
cur.close()
del cur

# Closing the connection -- IMPORTANT -- 
con.close()